9-6 패스트 텍스트(FastText)

Word2Vec는 단어를 쪼개질 수 없는 단위로 생각한다면, FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주한다.\
내부단어 즉, 서브워드(subword)를 고려하여 학습한다.

1. 내부 단어의 학습

FastText에서는 각 단어는 글자 단위 n-gram의 구성으로 취급한다. 만약 n을 3으로 잡은 tri-gram의 경우\
apple은 app,ppl,ple로 분리하고 이를 벡터로 만든다. 정확히는 시작과 끝을 의미하는 <,>를 도입하여 5개 내부 단어 토큰을 벡터로 만든다.

In [ ]:
# n = 3인 경우
<ap, app, ppl, ple, le>

그리고 여기에 추가적으로 하나를 더 벡터화하는데, 이는 기존 단어에 < 와 > 를 붙인 토큰이다.

In [ ]:
# 추가 토큰
<apple>

In [ ]:
#즉, n = 3인 경우에 FastText는 단어 apple에 대해서 6개의 토큰을 벡터화 한다
<ap, app, ppl, ple, le>, <apple>

그런데 실제로 사용할 때에는 n의 최솟값과 최대값으로 설정할 수 있는데, 기본값으로는 각각 3과 6으로 설정되어져 있다.\
즉 최솟값 = 3, 최대값 = 6인 경우라면 단어 apple에 대해서 FastText는 아래 내부 단어들을 벡터화한다.

In [ ]:
# n = 3 ~ 6 인 경우
<ap, app, ppl, ple, le>, <app, appl, pple, ple>, <appl, pple>, ..., <apple>

내부 단어들을 벡터화한다는 의미는 저 단어들에 대해서 Word2Vec을 수행한다는 의미이다.\
내부 단어들의 벡터값을 얻었다면, 단어 apple의 벡터값은 저 위 벡터값들의 총 합으로 구성된다.

In [ ]:
apple = <ap + app + ppl + ppl + le> + <app + appl + pple + ple> + <appl + pple> + , ..., +<apple>

2. 모르는 단어(Out of Vocabulary, OOV)에 대한 대응

FastText의 인공 신경망을 학습한 후 데이터 셋의 모든 단어의 각 n-gram에 대해서 임베딩 되는데, 이렇게 되면 데이터 셋만 충분하면 위와 같은 내부 단어를 통해 모르는 단어(OOV)에 대해서도 다른 단어와의 유사도를 계산할수 있다.\
ex) FastText에서 birthplace(출생지)란 단어가 학습되지 않은 상태라고 가정하자. 하지만 다른 단어에서 birth와 place라는 내부 단어가 있었다면, FastText는 birthplace라는 벡터를 얻을 수 있다.

3. 단어 집합 내 빈도 수가 적었던 단어(Rare Word)에 대한 대응

Word2Vec의 경우에는 등장 빈도수가 적은 단어(rare word)에 대해서는 임베딩 정확도가 높지 않다는 단점이 존재.\
FastText의 경우 단어가 희귀 단어라도 그 단어의 n-gram이 다른 단어의 n-gram과 겹치는 경우라면, Word2Vec과 비교했을 때 비교적 높은 임베딩 벡터값을 얻는다.\
실제 데이터들에는 오타가 섞여있는 경우가 많고 이로인해 등장 빈도수가 적어져 희귀 단어 처리가 되는 경우가 많은데, FastText는 이에 대해서도 일정 수준의 성능을 보인다.

4. 실습으로 비교하는 Word2Vec Vs. FastText

1) Word2Vec

In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model_wv = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [2]:
model_wv.most_similar("electrofishing")

KeyError: "Key 'electrofishing' not present in vocabulary"

단어 집합에 electrofishing이 존재하지 않아서 에러가 발생한다.

2) FastText

In [7]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
from gensim.models import FastText

targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower()) # a-z , 0-9 제와 나머지 공백처리인듯
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model.wv.most_similar("electrofishing")

5. 한국어에서의 FastText

(1) 음절 단위

예를 들어서 음절 단위의 임베딩의 경우에 n=3 일때 '자연어처리'라는 단어에 대해 n-gram을 만들어보면 다음과 같다.

In [ ]:
<자연, 자연어, 연어처, 어처리, 처리>

(2) 자모 단위

음절 단위가 아니라, 자모 단위로 가게 되면 오타나 노이즈 측면에서 더 강한 임베딩을 기대해볼 수 있다.\
예를 들어 '자연어처리'라는 단어에 대해서 초성,중성,종성을 분리하고 만약 종성이 존재하지 않는다면 '-'라는 토큰을 사용한다고 가정하면 아래와 같이 분리된다.

In [ ]:
ㅈ ㅏ _ ㅇ ㅕ ㄴ ㅇ ㅓ _ ㅊ ㅓ _ ㄹ ㅣ _